In [70]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize 
from data_retriever import read_tickers

### Format Curated Data

In [118]:
# read in data
tickers = ['AAPL', 'AMZN', 'GOOGL', 'FB']
df1 = pd.DataFrame()

for symbol in tickers:
    try:
        adj_close = pd.read_csv(f'data/time_series_daily_curated/{symbol}.csv', index_col=0).loc['5. adjusted close',:]
        adj_close.name = symbol
        df1 = df.append(adj_close)
    except:
        continue
        
df1 = df1.transpose()

# convert index to datetime
df1.index = pd.to_datetime(df1.index, infer_datetime_format=True)
df1 = df1.loc['2016-01-01':'2017-12-31']

df1

,AAPL,AMZN,GOOGL,FB
2016-01-04,24.320249,636.99,759.44,102.22
2016-01-05,23.710800,633.79,761.53,102.73
2016-01-06,23.246788,632.65,759.33,102.97
2016-01-07,22.265667,607.94,741.00,97.92
2016-01-08,22.383401,607.05,730.91,97.33
...,...,...,...,...
2017-12-22,41.967723,1168.36,1068.86,177.20
2017-12-26,40.903003,1176.76,1065.85,175.99
2017-12-27,40.910197,1182.26,1060.20,177.62
2017-12-28,41.025302,1186.10,1055.95,177.92


In [139]:
# read in data
tickers = ['AAPL', 'NKE', 'GOOGL', 'AMZN']
df2 = pd.DataFrame()

for symbol in tickers:
    try:
        adj_close = pd.read_csv(f'data/time_series_daily_curated/{symbol}.csv', index_col=0).loc['5. adjusted close',:]
        adj_close.name = symbol
        df2 = df2.append(adj_close)
    except:
        continue
        
df2 = df2.transpose()

# convert index to datetime
df2.index = pd.to_datetime(df2.index, infer_datetime_format=True)
df2 = df2.loc['2015-01-01':'2019-12-31']

df2

,AAPL,NKE,GOOGL,AMZN
2015-01-02,24.817291,44.266137,529.55,308.52
2015-01-05,24.118149,43.553444,519.46,302.19
2015-01-06,24.120419,43.297248,506.64,295.29
2015-01-07,24.458640,44.191607,505.15,298.42
2015-01-08,25.398397,45.211736,506.91,300.46
...,...,...,...,...
2019-12-24,70.233925,98.801833,1344.43,1789.21
2019-12-26,71.627386,99.364216,1362.47,1868.77
2019-12-27,71.600209,100.212724,1354.64,1869.80
2019-12-30,72.025165,99.453014,1339.71,1846.89


### Format Full Data

In [131]:
# read in data
tickers = read_tickers()['Symbol']
df = pd.DataFrame()

for symbol in tickers:
    try:
        adj_close = pd.read_csv(f'data/time_series_daily/{symbol}.csv', index_col=0).loc['5. adjusted close',:]
        adj_close.name = symbol
        df = df.append(adj_close)
    except:
        continue
        
df = df.transpose()

# convert index to datetime
df.index = pd.to_datetime(df.index, infer_datetime_format=True)

df

,ABM,ACN,AER,AI,AL,ALTG,AMWL,ANET,ARW,ASAN,...,WORK,WPP,XRX,YALA,YELP,YEXT,ZEN,ZEPP,ZIP,ZUO
2021-01-08,40.598779,262.446892,44.40,143.63,43.179891,10.37,27.14,306.12,104.35,31.58,...,42.35,56.44,23.312121,15.19,32.53,17.12,149.37,13.34,NaN,14.37
2021-01-11,40.558928,258.661600,45.76,135.62,44.804619,10.38,26.36,302.96,104.00,32.08,...,42.07,55.17,23.203232,17.71,32.31,16.60,152.26,13.57,NaN,14.23
2021-01-12,41.475514,256.724246,46.35,133.98,45.502355,10.51,26.29,316.28,104.77,34.98,...,42.14,55.56,22.985455,17.79,31.95,16.83,152.66,13.53,NaN,14.34
2021-01-13,41.575143,256.287099,44.74,143.14,44.037109,10.37,25.25,309.63,103.69,36.48,...,42.29,54.33,22.708283,18.10,31.40,16.75,147.11,13.69,NaN,14.08
2021-01-14,41.893955,254.213515,46.48,138.50,45.083713,10.09,28.70,313.10,104.11,37.14,...,41.92,55.43,22.807273,17.34,31.87,16.91,146.23,14.04,NaN,13.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-26,49.740000,283.950000,58.91,61.43,47.110000,14.40,12.12,338.46,121.96,36.10,...,43.13,69.37,23.750000,16.05,38.73,12.53,136.45,10.00,21.10,15.04
2021-05-27,50.010000,280.050000,59.00,63.20,47.080000,14.57,12.50,337.12,121.24,36.84,...,42.85,69.20,23.990000,15.94,39.81,12.31,137.05,10.37,20.85,15.06
2021-05-28,49.890000,282.160000,59.00,61.71,47.060000,14.66,12.45,339.38,120.33,36.78,...,44.04,69.19,23.450000,15.57,40.11,14.47,136.66,9.99,21.01,15.47
2021-06-01,50.660000,280.880000,60.11,71.76,48.200000,14.54,12.83,334.87,121.94,37.00,...,43.84,71.33,23.420000,17.88,39.93,13.68,134.62,10.43,23.05,14.98


### Analysis & Metrics

In [132]:
# expected daily returns
expected_returns = df.pct_change().mean()

# log of percent change
pct_change = df.pct_change().apply(lambda x : np.log(1+x))

# correlation matrix
corr_matrix = pct_change.corr()

# covariance matrix
cov_matrix = pct_change.cov()
cov_matrix

,ABM,ACN,AER,AI,AL,ALTG,AMWL,ANET,ARW,ASAN,...,WORK,WPP,XRX,YALA,YELP,YEXT,ZEN,ZEPP,ZIP,ZUO
ABM,0.000399,0.000104,0.000216,0.000119,0.000196,0.000259,-0.000011,0.000057,0.000159,0.000006,...,4.657599e-07,0.000135,0.000184,-0.000045,0.000238,0.000034,-0.000008,-0.000047,0.000243,0.000043
ACN,0.000104,0.000157,0.000096,0.000037,0.000086,0.000092,-0.000103,0.000102,0.000110,0.000099,...,4.406658e-05,0.000083,0.000064,0.000142,0.000065,0.000160,0.000097,0.000032,0.000045,0.000128
AER,0.000216,0.000096,0.000865,0.000046,0.000624,0.000227,-0.000128,0.000079,0.000183,-0.000021,...,-1.030174e-05,0.000200,0.000272,0.000229,0.000234,-0.000012,0.000073,0.000058,0.000352,0.000083
AI,0.000119,0.000037,0.000046,0.003062,0.000128,0.000442,0.001110,0.000207,0.000302,0.000742,...,9.165089e-05,0.000123,0.000189,0.001894,0.000487,0.000886,0.000373,0.000859,0.002965,0.000530
AL,0.000196,0.000086,0.000624,0.000128,0.000670,0.000219,-0.000155,0.000073,0.000150,0.000056,...,-3.919406e-06,0.000163,0.000216,0.000223,0.000188,0.000030,0.000108,0.000046,0.000501,0.000047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YEXT,0.000034,0.000160,-0.000012,0.000886,0.000030,0.000353,0.000650,0.000334,0.000220,0.000672,...,1.736614e-04,0.000088,0.000083,0.000917,0.000314,0.001456,0.000518,0.000737,-0.002105,0.000718
ZEN,-0.000008,0.000097,0.000073,0.000373,0.000108,0.000240,0.000149,0.000209,0.000110,0.000554,...,1.028553e-04,0.000049,0.000082,0.000736,0.000009,0.000518,0.000611,0.000392,-0.000355,0.000454
ZEPP,-0.000047,0.000032,0.000058,0.000859,0.000046,0.000409,0.001089,0.000243,0.000243,0.000866,...,1.373274e-04,0.000097,0.000115,0.001289,0.000243,0.000737,0.000392,0.001851,0.000655,0.000519
ZIP,0.000243,0.000045,0.000352,0.002965,0.000501,-0.000406,0.000234,-0.000275,0.000415,-0.000150,...,-2.108014e-04,0.000668,0.000020,0.003173,-0.000543,-0.002105,-0.000355,0.000655,0.002063,-0.000881


In [136]:
# returns expected_returns and cov_matrix
def extract_info(df):
    returns = df.pct_change().mean()
    cov = df.pct_change().apply(lambda x : np.log(1+x)).cov()
    return returns, cov

### Efficient Frontier

In [129]:
RISK_FREE_RATE = 0.0162 # current risk free rate
NUM_TRADING_DAYS = 250


# annualised performance of given weights
def annualised_performance(weights, expected_returns, cov_matrix):
    returns = np.dot(weights, expected_returns) * NUM_TRADING_DAYS
    volatility = np.sqrt(cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum() * NUM_TRADING_DAYS)
    return returns, volatility
    

# sharpe ratio
def sharpe_ratio(weights, expected_returns, cov_matrix):
    returns, volatility = annualised_performance(weights, expected_returns, cov_matrix)
    return (returns - RISK_FREE_RATE) / volatility

Sharpe ratio: 1.4601746433880298


In [135]:
# negative sharpe ratio
def neg_sharpe_ratio(weights, expected_returns, cov_matrix):
    return -sharpe_ratio(weights, expected_returns, cov_matrix)


# optimize returns for a target volatility
def optimize_sharpe_ratio(expected_returns, cov_matrix):
    num_assets = len(expected_returns) 
    args = (expected_returns, cov_matrix)
    bounds = tuple((0.0, 1.0) for asset in range(num_assets))
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    return minimize(neg_sharpe_ratio, num_assets * [1/num_assets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)

    
# optimize volatility for a target return
def optimize_volatility():
    pass

In [141]:
# random weights
weights = np.random.random(len(expected_returns))
weights = weights / np.sum(weights)
print(f'Sharpe ratio (random): {sharpe_ratio(weights, expected_returns, cov_matrix)}')

# optimized weights
expected_returns, cov_matrix = extract_info(df2)
result = optimize_sharpe_ratio(expected_returns, cov_matrix).x
returns, volatility = annualised_performance(result, expected_returns, cov_matrix)
print(f'Sharpe ratio (optimized): {sharpe_ratio(result, expected_returns, cov_matrix)}')
print(f'Annualised returns: {returns}')
print(f'Annualised volatility: {volatility}')
print(f'Weights: {result}')

Sharpe ratio (random): 1.2508706792975497
Sharpe ratio (optimized): 1.3789524348839826
Annualised returns: 0.33007897601257574
Annualised volatility: 0.22762132186160852
Weights: [1.86629545e-01 1.88372754e-01 3.87602277e-17 6.24997701e-01]


### References
https://www.machinelearningplus.com/machine-learning/portfolio-optimization-python-example/